# Python

## Further reading

https://techdevguide.withgoogle.com/
https://github.com/vinta/awesome-python



### Nested Functions
Because of the first-class nature of functions in Python, you can define functions inside other functions. Such functions are called nested functions. The child functions are only in scope inside the parent function and cannot be called outside.

In [3]:
def parent():
    print("Printing from the parent() function.")

    def first_child():
        return "Printing from the first_child() function."

    def second_child():
        return "Printing from the second_child() function."

    print(first_child())
    print(second_child())
    pass

parent()

Printing from the parent() function.
Printing from the first_child() function.
Printing from the second_child() function.


## Decorators

Decorators allow you to inject or modify code in functions or classes. The @ indicates the application of the decorator.


### Function decorators

A function decorator is applied to a function definition by placing it on the line before that function definition begins.

```python
# PythonDecorators/my_decorator.py
class my_decorator(object):

    def __init__(self, f):
        print("inside my_decorator.__init__()")
        f() # Prove that function definition has completed

    def __call__(self):
        print("inside my_decorator.__call__()")

@my_decorator
def aFunction():
    print("inside aFunction()")

print("Finished decorating aFunction()")

aFunction()
```
When you run this code, you see:
```python
inside my_decorator.__init__()
inside aFunction()
Finished decorating aFunction()
inside my_decorator.__call__()
```

The @ is just a little syntax sugar meaning “pass a function object through another function and assign the result to the original function.”  It brings the idea of “applying code to other code” (i.e.: macros) into mainstream thinking by formalizing it as a language construct.


### Extended example

```python
# PythonDecorators/entry_exit_class.py
class entry_exit(object):

    def __init__(self, f):
        self.f = f

    def __call__(self):
        print("Entering", self.f.__name__)
        self.f()
        print("Exited", self.f.__name__)

@entry_exit
def func1():
    print("inside func1()")

@entry_exit
def func2():
    print("inside func2()")

func1()
func2()
```

The output is:

```python
Entering func1
inside func1()
Exited func1
Entering func2
inside func2()
Exited func2
```

### Decorators with Arguments

When decorators are specified with arugements, e.g.:

```python
# PythonDecorators/decorator_with_arguments.py
class decorator_with_arguments(object):

    def __init__(self, arg1, arg2, arg3):
        """
        If there are decorator arguments, the function
        to be decorated is not passed to the constructor!
        """
        print("Inside __init__()")
        self.arg1 = arg1
        self.arg2 = arg2
        self.arg3 = arg3

    def __call__(self, f):
        """
        If there are decorator arguments, __call__() is only called
        once, as part of the decoration process! You can only give
        it a single argument, which is the function object.
        """
        print("Inside __call__()")
        def wrapped_f(*args):
            print("Inside wrapped_f()")
            print("Decorator arguments:", self.arg1, self.arg2, self.arg3)
            f(*args)
            print("After f(*args)")
        return wrapped_f

@decorator_with_arguments("hello", "world", 42)
def sayHello(a1, a2, a3, a4):
    print('sayHello arguments:', a1, a2, a3, a4)
```

the process of decoration calls the constructor and then immediately invokes __call__(). The arugments are fed to the constructor of the decorator.

### Conceptual

An explicit example of the concept of a decorator in Python (not using the @ syntax):



In [4]:
def my_decorator(some_function):

    def wrapper():

        print("Something is happening before some_function() is called.")

        some_function()

        print("Something is happening after some_function() is called.")

    return wrapper


def just_some_function():
    print("Wheee!")


just_some_function = my_decorator(just_some_function)

just_some_function()


Something is happening before some_function() is called.
Wheee!
Something is happening after some_function() is called.


Decorators in Python can be called via the 'pie' syntax: <i>@</i>, which is shorthand that enables <i>@my_decorator</i> to be used rather than: <i>just_some_function = my_decorator(just_some_function)</i>. 

Example timing decorator:

In [5]:
import time


def timing_function(some_function):

    """
    Outputs the time a function takes
    to execute.
    """

    def wrapper():
        t1 = time.time()
        some_function()
        t2 = time.time()
        return "Time it took to run the function: " + str((t2 - t1)) + "\n"
    return wrapper


@timing_function
def my_function():
    num_list = []
    for num in (range(0, 10000)):
        num_list.append(num)
    print("\nSum of all the numbers: " + str((sum(num_list))))


print(my_function())


Sum of all the numbers: 49995000
Time it took to run the function: 0.0030024051666259766



### Property decorator

In Python, property() is a built-in function that creates and returns a property object. The signature of this function is:

```property(fget=None, fset=None, fdel=None, doc=None)```

Example use of @property decorator:

In [ ]:
class Celsius:
    def __init__(self, temperature = 0):
        self._temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

    @property
    def temperature(self):
        print("Getting value")
        return self._temperature

    @temperature.setter
    def temperature(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        print("Setting value")
        self._temperature = value

## Multithreading and Multiprocessing

In Python the Global Interpreter Lock (GIL), a mechanism to synchronize the execution of threads such that only one native thread can execute at a time, means that code that's CPU-bound doesn't scale across threads, I/O bound code may however scale to some extent.

<i>multiprocessing</i> is a package that supports spawning processes using an API similar to the threading module. The multiprocessing package offers both local and remote concurrency, effectively side-stepping the Global Interpreter Lock by using subprocesses instead of threads. Due to this, the multiprocessing module allows the programmer to fully leverage multiple processors on a given machine.

In [ ]:
from multiprocessing import Pool
import time

def sum_prime(num):    
    sum_of_primes = 0
    ix = 2
    while ix <= num:
        if is_prime(ix):
            sum_of_primes += ix
        ix += 1
    return sum_of_primes

def is_prime(num):
    if num <= 1:
        return False
    elif num <= 3:
        return True
    elif num%2 == 0 or num%3 == 0:
        return False
    i = 5
    while i*i <= num:
        if num%i == 0 or num%(i+2) == 0:
            return False
        i += 6
    return True

if __name__ == '__main__':
    start = time.time()
    with Pool(1) as p:
        print(p.map(sum_prime, [1000000, 2000000, 3000000]))
    print("Time taken = {0:.5f}".format(time.time() - start))

## Strings
The ‘r‘ right before a string means “raw,” which takes the backslashes literally so you don’t have to put in an extra backslash in order to insert a literal backslash.
```python
print(r'c:\python\lib\utils')
```

## Main tags

```python
if __name__ == "__main__":
    # Create an object:
    x = Simple("constructor argument")
```

This particular if statement is only true when you are running this file directly; that is, if you say on the command line:

```python
Python SimpleClass.py

```
However, if this file is imported as a module into another program, the __main__ code is not executed.

## Inheritance

Because Python is dynamically typed, it doesn’t really care about interfaces - all it cares about is applying operations to objects (in fact, Java’s interface keyword would be wasted in Python). In Python, the only reason you inherit is to inherit an implementation - to re-use the code in the base class.

## Expanding

You can turn a list into function arguments using *:
```python
def f(a,b,c): print a, b, c
x = [1,2,3]
f(*x)
f(*(1,2,3))

```
If the form ```*identifier``` is present, it is initialized to a tuple receiving any excess positional parameters, defaulting to the empty tuple. If the form ```**identifier``` is present, it is initialized to a new dictionary receiving any excess keyword arguments, defaulting to a new empty dictionary.

# Tuples vs Lists

Tuples are fixed size in nature whereas lists are dynamic.<br>
In other words, a tuple is immutable whereas a list is mutable.

- Tuples are faster than lists. If you're defining a constant set of values and all you're ever going to do with it is iterate through it, use a tuple instead of a list.

- It makes your code safer if you “write-protect” data that does not need to be changed. Using a tuple instead of a list is like having an implied assert statement that this data is constant, and that special thought (and a specific function) is required to override that.

- Some tuples can be used as dictionary keys (specifically, tuples that contain immutable values like strings, numbers, and other tuples). Lists can never be used as dictionary keys, because lists are not immutable.